This notebook uses the **cleaned data** loaded into `ecommerce_ops.db` to answer key business questions about:

1. Find the top 10 customers by total revenue.
2. Calculate the Average Order Value(AOV) from delivered orders only
3. Which products have generated more than $1,000 in total net revenue?
4. List all customers who have placed ≥ 3 orders in the last 90 days.
5. For each product category, calculate total units sold and total revenue.
6. Which warehouse has the fastest average delivery time?
7. Identify all orders that had discounts applied AND were later returned.
8. For each customer, calculate their most recent order date.
9. What percentage of orders were cancelled in the last 60 days?
10. Find the top 3 carriers by on-time delivery rate (delivered within 3 days of ship date).

Each section follows this structure:

- **SQL query** to pull/aggregate data  
- **Result table** (and possibly a chart)  
- **Short interpretation** of what the results mean

In [12]:
import pandas as pd
import sqlite3

# Connect to the cleaned SQLite database
conn = sqlite3.connect(
    "../01_ecommerce_ops_analytics/data/processed/ecommerce_ops.db"
)

# Optional: show tables available
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,customers
1,inventory
2,orders
3,products
4,returns
5,shipments
6,order_items


## 2. Quick Data Sanity Check

Before diving into analysis, confirm that key tables look correct and contain data.
We only show a couple of checks here because detailed cleaning was done in `02_data_cleaning.ipynb`.

In [13]:
tables = ["customers", "orders", "order_items", "products", "inventory", "returns", "shipments"]

for t in tables:
    count = pd.read_sql(f"SELECT COUNT(*) AS rows FROM {t};", conn)
    print(t)
    display(count)

customers


,rows
0,75


orders


,rows
0,3000


order_items


...

,rows
0,120


returns


,rows
0,5


shipments


,rows
0,2549


## 3. Customer Analysis

In this section we look at customer-level behavior:
- Who are the top 10 customers by total revenue?
- How many orders have they placed >= 3 orders in the last 90 days?
- When was their most recent order?

In [14]:
revenue_query = """
SELECT
    customer_id,
    first_name,
    last_name,
    total_spent AS total_revenue
FROM customers
ORDER BY total_revenue DESC
LIMIT 10;
"""
top_customers_revenue = pd.read_sql(revenue_query, conn)
top_customers_revenue

,customer_id,first_name,last_name,total_revenue
0,33,Kayla,Wright,5198.24
1,4,Sierra,Anderson,4473.08
2,71,Aaliyah,Johnson,4114.53
3,65,Jordan,Davis,3879.64
4,14,Tiana,Taylor,3798.53
5,16,Chris,Davis,3746.48
6,6,Brianna,White,3743.82
7,34,Sierra,Green,3411.65
8,10,Tiana,Scott,3319.27
9,68,Terrence,Anderson,3258.00


In [15]:
recent_orders_query = """
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    COUNT(o.order_id) AS orders_last_90_days
FROM customers AS c
JOIN orders AS o
    ON c.customer_id = o.customer_id
WHERE o.order_date >= DATE('now', '-90 days')
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING COUNT(o.order_id) >= 3
ORDER BY orders_last_90_days DESC
LIMIT 10;

"""
frequent_customers = pd.read_sql(recent_orders_query, conn)
frequent_customers

,customer_id,first_name,last_name,orders_last_90_days
0,10,Tiana,Scott,6
1,11,Kayla,Wright,5
2,22,Jordan,Taylor,5
3,3,Aaliyah,Jackson,4
4,47,Chris,Brown,4
5,48,Terrence,Jackson,4
6,50,Devon,Adams,4
7,56,Jasmine,King,4
8,58,Andre,Thomas,4
9,66,Alicia,Moore,4


In [16]:
most_recent_order_query = """
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    MAX(o.order_date) AS most_recent_order_date
FROM customers AS c
JOIN orders AS o
    ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY most_recent_order_date DESC
LIMIT 10;
"""
most_recent_orders = pd.read_sql(most_recent_order_query, conn)
most_recent_orders

,customer_id,first_name,last_name,most_recent_order_date
0,67,Danielle,Thomas,2025-11-12 15:22:00
1,59,Nia,Brown,2025-11-12 00:43:00
2,23,Marcus,Scott,2025-11-11 22:16:00
3,10,Tiana,Scott,2025-11-11 07:38:00
4,54,Chris,Moore,2025-11-10 05:02:00
5,63,Jasmine,Wright,2025-11-09 04:15:00
6,53,Nia,Jackson,2025-11-08 06:39:00
7,50,Devon,Adams,2025-11-07 20:09:00
8,11,Kayla,Wright,2025-11-07 18:12:00
9,44,Jordan,Moore,2025-11-07 16:19:00


## 3. Customer Analysis

### 3.1 Top Customers by Revenue
- The top-spending customer is Kayla Wright (customer_id 33) with $5,198.24 in total revenue.

### 3.2 Frequent Orders from Top Customers
- In the recent-orders analysis, another customer also named Kayla Wright (customer_id 11) appears among the most active buyers, but they are different individuals.

### 3.3 Most Recent Orders from Top Customers
-Kayla (customer_id 11) most recently purchased on 11-07-2025 but we have been getting between 2-3 orders per day from these customers.

### Analysis
These customers would be good candidates for loyalty programs, early access, or targeted promotions.
Marketing could increase revenue by targeting these high-value customers with personalized offers.

## 3. Shipment Analysis

In this section we look at logistics:
- Which warehouse has the fastest average delivery time?
- Find the top 3 carriers by on-time delivery rate (delivered within 3 days of ship date).

In [17]:
shipments_speed_query = """
Select
    s.warehouse,
    AVG(julianday(s.delivered_date) - julianday(s.shipped_date)) AS avg_delivery_days
FROM shipments AS s
WHERE s.delivered_date IS NOT NULL
GROUP BY s.warehouse
ORDER BY avg_delivery_days ASC;    

"""
fastest_warehouses = pd.read_sql(shipments_speed_query, conn)
fastest_warehouses


,warehouse,avg_delivery_days
0,DFW-02,2.903039
1,ATL-01,2.989394
2,CLT-03,3.024259


In [18]:
top_carriers_query = """
SELECT
    s.carrier,
    COUNT(*) AS total_shipments,
    SUM(CASE 
            WHEN julianday(s.delivered_date) - julianday(s.shipped_date) <= 3 
            THEN 1 
            ELSE 0 
        END) AS on_time_deliveries,
    ROUND(SUM(CASE 
            WHEN julianday(s.delivered_date) - julianday(s.shipped_date) <= 3 
            THEN 1 
            ELSE 0 
        END) * 100.0 / COUNT(*), 2) AS on_time_delivery_rate
FROM shipments AS s
WHERE s.delivered_date IS NOT NULL
GROUP BY s.carrier
ORDER BY on_time_delivery_rate DESC
LIMIT 3;
"""
top_carriers = pd.read_sql(top_carriers_query, conn)
top_carriers

,carrier,total_shipments,on_time_deliveries,on_time_delivery_rate
0,DHL,562,361,64.23
1,USPS,537,341,63.50
2,UPS,508,302,59.45


## 3. Shipment Analysis

### 3.1 Fastest Warehouse
 - DFW-02 is the fastest warehouse with a 0.1 avg delivery difference over the slowest warehouse(CLT-03)
 

### 3.2 Fastest Carriers by Delivery
- DHL is the fastest carrier followed by USPS then UPS. DHL has 361 on time deliveries while our lowest carrier(UPS) has 302. 

### Anaylsis
- We can look into why DFW-02 is the fastest warehouse to see if they may be doing something different than CLT-03. We could implement that difference into ATL-01 and CLT-03.

- We could take some load off UPS and send it USPS since DHL already is handling most of the shipments.

## 4. Product Analysis

- Which products have generated more than $1,000 in total net revenue?
- For each product category, calculate total units sold and total revenue.

In [19]:
product_revenue_query = """
SELECT
    p.product_id,
    p.product_name,
    SUM(oi.quantity * oi.unit_price - oi.discount) AS total_net_revenue
FROM order_items AS oi
JOIN products AS p
    ON oi.product_id = p.product_id
GROUP BY p.product_id, p.product_name
HAVING total_net_revenue > 1000
ORDER BY total_net_revenue DESC;
"""
high_revenue_products = pd.read_sql(product_revenue_query, conn)
high_revenue_products

,product_id,product_name,total_net_revenue
0,55,Electronics Item 55,1814.04
1,30,Electronics Item 30,1729.16
2,71,Electronics Item 71,1376.34
3,109,Electronics Item 109,1004.69


In [20]:
units_sold_query = """
SELECT
    p.category,
    SUM(oi.quantity) AS total_units_sold,
    SUM(oi.quantity * oi.unit_price - oi.discount) AS total_revenue
FROM order_items AS oi
JOIN products AS p
    ON oi.product_id = p.product_id
GROUP BY p.category
ORDEr BY total_units_sold DESC;
"""
category_sales = pd.read_sql(units_sold_query, conn)
category_sales

,category,total_units_sold,total_revenue
0,Beauty,76,4351.55
1,Sports,66,10936.73
2,Toys,56,5365.71
3,Home,47,5838.33
4,Electronics,46,13635.39
5,Grocery,32,702.15


## 4 Product Analysis

### 4.1 Product Revenue
-   Electronics Item 55 has made the company the most money with $1,814.04

### 4.2 Unit Sold, Revenue
-   Beauty products sold the most at 76 but it was second last in total revenue. Electronics were the highest with $13,635.39 and Sports at $10,936.73.

### Analysis 
-   Maybe we can test the market to test raising prices to make more money compared to other categories or we can push electronics and sports more to increase their revenue.

- Electronics has the top products that have brought in the most revenue. We could explore doubling down on Electronics or get people to look more towards other departments to help raise the overall revenue.
    



## 5. Order Analysis

-  Calculate the Average Order Value (AOV) from delivered orders only.
-  Identify all orders that had discounts applied AND were later returned.
-  What percentage of orders were cancelled in the last 60 days?

In [36]:
aov_query = """
SELECT
    AVG(order_totals.order_revenue) AS average_order_value
FROM (
    SELECT
        o.order_id,
        SUM(oi.quantity * oi.unit_price - oi.discount) AS order_revenue
    FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_id
    WHERE o.status = 'delivered'
    GROUP BY o.order_id
) AS order_totals;
"""
aov_result = pd.read_sql(aov_query, conn)
aov_result

print(f"Average Order Value (AOV) from delivered orders: ${aov_result['average_order_value'][0]:,.2f}")

Average Order Value (AOV) from delivered orders: $332.33


In [37]:
disounted_returned_orders_query = """
SELECT
    o.order_id,
    o.customer_id,
    o.order_date,
    oi.discount,
    r.return_id,
    r.return_date,
    r.refund_amount
FROM orders AS o
JOIN order_items AS oi
    ON o.order_id = oi.order_id
JOIN returns AS r
    ON o.order_id = r.order_id
WHERE oi.discount > 0;
"""
discounted_returned_orders = pd.read_sql(disounted_returned_orders_query, conn)
discounted_returned_orders

,order_id,customer_id,order_date,discount,return_id,return_date,refund_amount
0,12,147,2025-04-25 01:55:00,0.05,2,2025-05-08,39.23


In [38]:
cancelled_orders_query = """
SELECT
    COUNT(*) AS cancelled_orders,
    (SELECT COUNT(*) FROM orders WHERE order_date >= DATE('now', '-60 days')) AS total_orders,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM  orders WHERE order_date >= DATE('now', '-60 days')), 2) AS cancelled_percentage
FROM orders
WHERE status = 'cancelled' AND order_date >= DATE('now', '-60 days');
"""
cancelled_orders_stats = pd.read_sql(cancelled_orders_query, conn)
cancelled_orders_stats

,cancelled_orders,total_orders,cancelled_percentage
0,60,657,9.13


### 5. Order Analysis

###     5.1 Delivered Order Values
    -   We had an average order value of $332.33 on orders that have been delivered.

###     5.2 Returned Discounted Products
    -   We only had one discounted product that has been returned. It was returned within two weeks as well.

###     5.3 Cancelled Orders
    -   We have had 60 cancelled orders from 657 total orders which is less than a 10% cancel percentage.

### Analysis
    - We are doing great with discounting products because we have only had one that was returned. So let's keep discounted products and/or tighten up a discount policy for that specific product if can not be used up within two weeks

    - We have a less than 10% cancel rate so we are also doing great in that area but engineering could make a pop-up when cancelling so we can know why they are cancelling.

### Summary

**Key Findings** 

- DFW-02 is the fastest warehouse with a .1 avg delivery date over the slowest warehouse(CLT-03)
- Revenue is primarily driven by Electronics.
- We only had one discounted product that has been returned.
- We had an average order value of $332.33 on orders that have been delivered.
- Beauty products sold the most at 76 but it was second last in total revenue. 